In [1]:
from langchain_aws import ChatBedrockConverse
from langchain.schema import HumanMessage, SystemMessage
import boto3
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from opensearchpy import OpenSearch, RequestsHttpConnection
import os
import boto3
import json
import sys
from langchain.schema import BaseRetriever, Document
from typing import List, Optional, Dict, Tuple
from langchain.prompts.prompt import PromptTemplate
import uuid
from datetime import datetime
from botocore.exceptions import ClientError
from operator import itemgetter
from langchain_core.runnables import RunnableLambda
from boto3.dynamodb.conditions import Key
from pydantic import BaseModel, Field
from langchain_core.output_parsers import JsonOutputParser
from langchain_core.runnables import RunnableParallel, RunnablePassthrough
from langchain_aws import ChatBedrock


In [2]:
print("langchain advanced")

langchain advanced


In [3]:
#bedrock client 설정
bedrock_client = boto3.client(
    service_name='bedrock-runtime',
    region_name='us-west-2'  # replace with your region
)

In [4]:
#llm
llm = ChatBedrock(
    client=bedrock_client,
    model_id="global.anthropic.claude-sonnet-4-20250514-v1:0",
    model_kwargs={
        "max_tokens": 2000,
        "temperature": 0.15,
        "top_p": 0.9,
    }
    )

In [5]:
async def extract_stream(chain, input_json):
    collected_text = []
    async for event in chain.astream_events(input_json):
        if event["event"] == "on_chat_model_stream":
            chunk = event["data"]["chunk"]
            if chunk.content:
                for content_item in chunk.content:
                    if text := content_item.get('text'):
                        collected_text.append(text)
                        print(text, end="", flush=True)
    return "".join(collected_text)


In [6]:
#input { json 포맷 }  -> prompt(text로 변환) -> llm(bedrock) -> output(string타입, json타입)
#여기서 input { json 포맷 }을 활용하는 방법에 대해 이해해보자 ( 여행에 관한 정보를 얻는 llm를 만든다고 가정한다 )

In [7]:
prompt_capital = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful assistant."),
    ("human", "{country}의 수도에 대해 알려줘")
])

capital_chain = prompt_capital | llm |  StrOutputParser()


# capital_chain.invoke({"country" : "태국"})

In [8]:
async for chunk in capital_chain.astream({"country": "태국"}):
    print(chunk, end="")

태국의 수도는 **방콕(Bangkok)**입니다.

## 방콕의 주요 정보

**정식 명칭**: 크룽텝마하나콘(กรุงเทพมหานคร)
- 태국어로는 매우 긴 정식 명칭이 있어 기네스북에 등재되기도 했습니다

**인구**: 약 1,000만 명 (수도권 포함)

**주요 특징**:
- 태국의 정치, 경제, 문화의 중심지
- 차오프라야 강이 도시를 관통
- 동남아시아의 주요 관광 도시

**유명한 관광지**:
- 왕궁(Grand Palace)
- 왓 포 사원(와불상으로 유명)
- 왓 아룬 사원(새벽 사원)
- 차이나타운
- 카오산 로드
- 수상시장들

**교통**: 
- BTS 스카이트레인
- MRT 지하철
- 툭툭, 택시, 보트 등 다양한 교통수단

방콕은 전통과 현대가 조화를 이루는 매력적인 도시로, 맛있는 음식과 풍부한 문화유산으로도 유명합니다.

In [9]:
prompt_culture = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful assistant."),
    ("human", "{country}의 문화에 대해 알려줘")
])

culture_chain = prompt_culture | llm |  StrOutputParser()


# culture_chain.invoke({"country" : "태국"})


In [10]:
async for chunk in culture_chain.astream({"country": "태국"}):
    print(chunk, end="")

태국의 풍부한 문화에 대해 소개해드리겠습니다.

## 🏛️ **종교와 철학**
- **불교**: 인구의 95%가 소승불교도
- 일상생활에 불교 철학이 깊이 스며들어 있음
- 전국에 4만여 개의 사원(왓)이 있음

## 🙏 **예의와 매너**
- **와이(Wai)**: 두 손을 모아 인사하는 전통 예법
- 연장자와 지위가 높은 사람에 대한 깊은 존경
- 온화하고 미소 짓는 문화 (미소의 나라)

## 🍜 **음식 문화**
- **쌀**: 주식이며 다양한 쌀 요리
- **톰얌꿍, 팟타이, 그린커리** 등 유명 요리
- 매운맛, 신맛, 단맛, 짠맛의 조화
- 길거리 음식 문화가 발달

## 🎭 **예술과 전통**
- **전통 무용**: 콘(Khon), 라콘(Lakhon) 등
- **무에타이**: 전통 격투기
- **태국 마사지**: 전통 치료법
- 정교한 사원 건축과 불상 조각

## 👑 **왕실 문화**
- 왕실에 대한 깊은 존경과 사랑
- 왕실 관련 예의가 매우 엄격
- 국왕 찬송가 연주 시 기립하는 전통

## 🎉 **축제와 기념일**
- **송크란(4월)**: 물축제, 태국 신정
- **로이 크라통(11월)**: 등불 축제
- **부처님 오신 날** 등 불교 축제들

태국 문화는 전통과 현대가 조화롭게 어우러진 독특한 매력을 가지고 있습니다.

In [11]:
prompt_climate = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful assistant."),
    ("human", "{country}의 날씨에 대해 알려줘")
])

climate_chain = prompt_climate | llm |  StrOutputParser()


# climate_chain.invoke({"country" : "태국"})

In [12]:
async for chunk in climate_chain.astream({"country": "태국"}):
    print(chunk, end="")

태국의 날씨에 대해 설명드리겠습니다.

## 기본 특징
- **열대성 기후**: 연중 고온다습한 날씨
- **평균 기온**: 25-35°C
- **습도**: 연중 높은 습도 (70-80%)

## 계절별 특징

### 🌞 건기 (11월-2월)
- 가장 여행하기 좋은 시기
- 기온: 25-30°C
- 습도가 상대적으로 낮음
- 강수량 적음

### 🔥 더운 건기 (3월-5월)
- 가장 더운 시기
- 기온: 30-40°C
- 매우 덥고 건조함
- 4-5월이 가장 더움

### 🌧️ 우기 (6월-10월)
- 몬순 시즌
- 기온: 25-32°C
- 거의 매일 스콜성 비
- 9-10월이 강수량 최고점

## 지역별 차이

### 북부 (치앙마이, 치앙라이)
- 상대적으로 서늘함
- 건기 때 일교차 큼 (밤에는 쌀쌀할 수 있음)

### 중부 (방콕)
- 전형적인 열대기후
- 도시 열섬효과로 더 더움

### 남부 (푸켓, 코사무이)
- 연중 고온다습
- 지역에 따라 우기 시기가 다름

## 여행 팁
- **최적 시기**: 11월-2월
- **준비물**: 가벼운 옷, 우산, 선크림, 모자
- **주의사항**: 우기에는 홍수 가능성 있음

In [13]:
prompt_location = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful assistant."),
    ("human", "{country}의 유명한 관광 명소에 대해 알려줘")
])

location_chain = prompt_location | llm |  StrOutputParser()


# climate_location.invoke({"country" : "태국"})

In [14]:
async for chunk in location_chain.astream({"country": "태국"}):
    print(chunk, end="")

태국의 주요 관광 명소들을 지역별로 소개해드리겠습니다.

## 🏛️ 방콕 (Bangkok)
- **왓 포 (Wat Pho)** - 거대한 와불상으로 유명한 사원
- **왓 아룬 (Wat Arun)** - 새벽 사원으로 불리는 아름다운 탑
- **그랜드 팰리스** - 태국 왕실의 화려한 궁전
- **차오프라야 강** - 롱테일 보트 투어
- **카오산 로드** - 배낭여행자들의 성지

## 🏖️ 푸켓 (Phuket)
- **파통 비치** - 나이트라이프와 해변 액티비티
- **피피 섬** - 영화 '더 비치' 촬영지
- **빅 부다** - 45m 높이의 거대한 불상
- **올드 타운** - 중국-포르투갈 스타일 건축물

## 🌴 코사무이 (Koh Samui)
- **차웽 비치** - 가장 인기 있는 해변
- **빅 부다 사원** - 12m 높이의 황금 불상
- **나무앙 폭포** - 열대우림 속 아름다운 폭포

## 🏛️ 치앙마이 (Chiang Mai)
- **도이 수텝 사원** - 산 위의 신성한 사원
- **올드 시티** - 고대 성벽과 전통 사원들
- **나이트 바자** - 전통 수공예품 쇼핑
- **코끼리 보호구역** - 윤리적 코끼리 체험

## 🏞️ 기타 명소
- **아유타야** - 유네스코 세계문화유산 고대 도시
- **수코타이** - 태국 최초 왕조의 유적지
- **크라비** - 석회암 절벽과 에메랄드 바다

각 지역마다 독특한 매력이 있으니, 관심사에 따라 선택하시면 됩니다!

In [15]:
#여행에 관한 정보를 얻는 llm를 만든다고 가정한다 위의 네가지 정보를 모두 포함하여 전체적인 형태의 여행 정보를 얻고 싶다고 가정 할때 어떻게 langchain에 활용할수 있을까?
#결국 llm에서 결과가 나오면 string임으로 이 string을 json안에 assgin가능하다

In [16]:
# input에 {"country": "태국"}이라고 두고 itemgetter("country")를 통해 변수를 가져온뒤 각각 llm의 결과를 얻고 sting으로 json에 포함시킨다면? 
# llm을 통과한 결과값들이 새로운 input(json타입)을 형성할수 있지 않을까?
new_input = {
                "capital" :  itemgetter("country") | capital_chain,
                "culture" :  itemgetter("country") | culture_chain,
                "climate" :  itemgetter("country") | climate_chain,
                "location":  itemgetter("country") | location_chain
            }

In [17]:
# new_input의 capital, culture, climate, location가  template_total의 capital, culture, climate, location 변수와 연결된다
template_total = """The following is a friendly conversation between a human and an AI. 
The AI is talkative and provides lots of specific details from its context. 

capitial :  {capital}
culture :   {culture}
climate :   {climate}
location :  {location}

이 정보를 바탕으로 여행 가기 가장 좋은 계절을 알려주세요
"""
prompt_total  = PromptTemplate(input_variables=["capital", "culture", "climate", "location"],
                                 template=template_total)

In [18]:
travel_chain = new_input | prompt_total | llm | StrOutputParser()

In [19]:
async for chunk in travel_chain.astream({"country": "태국"}):
    print(chunk, end="")

제가 제공한 정보를 종합해보면, **태국 여행하기 가장 좋은 계절은 11월부터 2월까지의 건기**입니다! 🌞

## 왜 이 시기가 최적일까요?

### 🌡️ 날씨 조건
- **기온**: 25-30°C로 가장 쾌적
- **습도**: 상대적으로 낮아서 덜 끈적거림
- **강수량**: 거의 비가 오지 않아 야외활동 최적
- **맑은 하늘**: 사진 찍기에도 완벽!

### 🏖️ 지역별 장점
**방콕**: 왕궁, 왓 포 등 야외 관광지 둘러보기 편함
**북부 (치앙마이)**: 도이 수텝 사원 트레킹하기 좋고, 밤에는 서늘해서 나이트 바자 구경하기 완벽
**남부 해변**: 푸켓, 크라비의 에메랄드빛 바다를 만끽하기 최고의 조건

### 🎉 문화적 보너스
- **12월-1월**: 태국의 겨울 성수기로 현지 축제들이 많아요
- **로이 크라통 축제** (11월): 등불 축제를 경험할 수 있는 특별한 시기

### 💡 여행 팁
다만 이 시기는 성수기라서:
- 항공료와 숙박비가 비쌀 수 있어요
- 인기 관광지가 붐빌 수 있으니 미리 예약 필수!
- 그래도 쾌적한 날씨 때문에 충분히 가치 있답니다

혹시 예산을 절약하고 싶으시다면 우기 직전인 **10월 말이나 우기 직후인 3월 초**도 괜찮은 대안이 될 수 있어요!

In [20]:
#비슷한 방법으로 RunnableParallel도 존재한다 
parallel_chain = RunnableParallel(
                                  capital=capital_chain, 
                                  culture=culture_chain,
                                  climate=climate_chain, 
                                  location =location_chain
                                 )

In [21]:
travel_chain_parallel =  parallel_chain | prompt_total | llm | StrOutputParser()

In [22]:
async for chunk in travel_chain_parallel.astream({"country": "태국"}):
    print(chunk, end="")


제공해주신 태국 정보를 바탕으로 말씀드리면, **11월부터 2월까지의 건기**가 태국 여행하기에 가장 좋은 계절입니다! 🌞

## 왜 이 시기가 최적일까요?

### 🌡️ **완벽한 날씨 조건**
- 기온: 25-30°C로 가장 쾌적함
- 습도가 상대적으로 낮아서 덜 끈적거림
- 강수량이 적어 야외활동하기 좋음
- 하늘이 맑아 사진 찍기에도 완벽!

### 🏖️ **지역별 장점**
- **방콕**: 왕궁, 왓 포, 왓 아룬 등 야외 관광지 둘러보기 최적
- **치앙마이**: 도이 수텝 트레킹이나 올드 시티 탐방하기 좋음  
- **푸켓/끄라비**: 파통 비치, 라일레이 비치에서 해양스포츠 즐기기 완벽
- **코사무이**: 차웽 비치에서 일광욕하기 이상적

### 🎉 **특별한 경험**
- **11월**: 로이 끄라통 등불 축제를 경험할 수 있어요!
- 차오프라야 강 크루즈나 피피 섬 투어 등 보트 투어하기에도 파도가 잔잔함

다만 이 시기는 성수기라서 숙박비가 비싸고 관광지가 붐빌 수 있어요. 그래도 태국의 아름다운 사원들과 해변을 가장 편안하게 즐길 수 있는 시기입니다! ✨

혹시 특정 지역이나 활동에 관심이 있으시다면 더 구체적인 조언을 드릴 수 있어요!

In [23]:
# llm의 outpu의 경우 StrOutputParser()를 활용해서 string으로 나오지만 llm.with_structured_output(obj)을 통해 json포맷으로 받을수 있다 
# llm의 결과가 json포맷으로 오면 이를 활용하여 다른 langchain에 활용할수 있다

In [24]:
template_plan = """The following is a friendly conversation between a human and an AI. 
The AI is talkative and provides lots of specific details from its context. 

capitial :  {capital}
culture :   {culture}
climate :   {climate}
location :  {location}

이 정보를 바탕으로 여행 계획을 세워주세요
"""
prompt_plan  = PromptTemplate(input_variables=["capital", "culture", "climate", "location"],
                                 template=template_plan)

In [25]:
#llm의 결과값 포맷의 schema
class Travel(BaseModel):
    capital: str = Field(description="국가의 수도에 대한 설명")
    culture: str = Field(description="국가의 문화에 대한 설명")
    climate : str = Field(description="국가 기후에 대한 설명")
    location : str = Field(description="국가 관광명소에 대한 설명")
    plan: str = Field(description="여행계획")


In [26]:
one_chain = (
    {
                "capital" :  itemgetter("country") | capital_chain,
                "culture" :  itemgetter("country") | culture_chain,
                "climate" :  itemgetter("country") | climate_chain,
                "location":  itemgetter("country") | location_chain
    }
    | prompt_plan
    | llm.with_structured_output(Travel)
)
 

In [27]:
async for chunk in one_chain.astream({"country": "태국"}):
    print(chunk, end="")


capital='태국의 수도는 방콕(Bangkok)입니다. 정식 명칭은 크룽텝마하나콘이며, 인구 약 1,000만 명의 대도시입니다. 태국의 정치, 경제, 문화의 중심지로 왕궁, 왓 포 사원, 왓 아룬 사원, 카오산 로드 등 주요 관광명소가 있습니다. 스카이트레인(BTS), 지하철(MRT), 보트 등 다양한 교통수단이 발달되어 있으며, 팟타이, 똠얌꿍 등 유명한 태국 요리를 맛볼 수 있습니다.' culture="태국은 인구의 95%가 소승불교도인 불교 국가로, 일상생활에 불교 철학이 깊이 스며들어 있습니다. 와이(Wai) 합장 인사법과 연장자 공경 문화가 있으며 '미소의 나라'로 불립니다. 쌀이 주식이고 매운맛, 신맛, 단맛, 짠맛이 조화된 음식문화가 발달했습니다. 무에타이 전통 무술, 우아한 전통 무용, 왕실에 대한 깊은 존경 문화가 있으며, 송크란(물축제), 로이 크라통(등불축제) 등 다양한 축제가 있습니다." climate='태국은 열대성 기후로 연중 고온다습하며 평균 기온은 25-35°C입니다. 건기(11월-2월)는 가장 여행하기 좋은 시기로 기온 25-30°C이며, 더운 건기(3월-5월)는 30-40°C로 가장 덥고, 우기(6월-10월)는 25-32°C로 몬순 시즌입니다. 북부는 상대적으로 서늘하고, 중부는 전형적인 열대기후, 남부는 연중 습하고 따뜻합니다. 최적 여행시기는 11월-2월입니다.' location="방콕: 왓 포(거대한 와불상), 왓 아룬(새벽 사원), 그랜드 팰리스(왕실 궁전), 차오프라야 강, 카오산 로드. 푸켓: 파통 비치, 피피 섬(영화 '더 비치' 촬영지), 빅 부다, 올드 타운. 끄라비: 라일레이 비치(암벽등반), 에메랄드 풀(천연 온천), 타이거 케이브 템플. 치앙마이: 왓 프라탓 도이 수텝(산 위의 황금 사원), 올드 시티, 나이트 바자. 코사무이: 빅 부다 템플, 나무앙 폭포, 차웽 비치." plan='태국 여행 계획을 세워드리겠습니다! \n\n**추천 일정 (7-10일)**:\n1일차: 방콕 도착 - 그랜드 팰리스, 왓 

In [28]:
#data는 llm의 결과 값을 의미합니다  RunnableLambda함수를 활용하면 data와 llm결과를 연결 시켜줄수 있습니다
def transform_keys(data):
    """Transform keys from history to chat_history"""
    return {
        "capital":  data.capital,
        "culture":  data.culture,  
        "climate":  data.climate, 
        "location": data.location, 
        "plan":     data.plan
    }

In [29]:
template_detail = """The following is a friendly conversation between a human and an AI. 
The AI is talkative and provides lots of specific details from its context. 

capitial :  {capital}
culture :   {culture}
climate :   {climate}
location :  {location}
plan :      {plan}

plan에 담겨있는 관광지 혹은 축제에 대한 정보를 좀더 상세하게 알려두세요 
"""
prompt_detail  = PromptTemplate(input_variables=["capital", "culture", "climate", "location", "plan"],
                                 template=template_detail)

In [32]:
# RunnableLambda(transform_keys)는 llm과 prompt_detail의 데이터를 매핑 시켜주는 역할
chain_and_chain = (
    {
                "capital" :  itemgetter("country") | capital_chain,
                "culture" :  itemgetter("country") | culture_chain,
                "climate" :  itemgetter("country") | climate_chain,
                "location":  itemgetter("country") | location_chain
    }
    | prompt_plan
    | llm.with_structured_output(Travel)
    | RunnableLambda(transform_keys)
    | prompt_detail
    | llm
    | StrOutputParser()
)


In [33]:
async for chunk in chain_and_chain.astream({"country": "일본"}):
    print(chunk, end="")


안녕하세요! 일본 여행 계획에 포함된 관광지와 축제에 대해 더 자세히 알려드리겠습니다.

## 🌸 봄 (벚꽃 시즌) - 도쿄-교토-오사카 골든 루트

**도쿄의 벚꽃 명소:**
- **우에노 공원**: 약 1,200그루의 벚나무가 있어 하나미(꽃구경) 파티의 성지입니다
- **치도리가후치**: 황궁 주변의 해자를 따라 벚꽃이 터널을 이루며, 야간 조명도 아름답습니다
- **신주쿠 교엔**: 65종류의 벚나무가 있어 오랜 기간 꽃구경이 가능합니다

**교토의 벚꽃과 축제:**
- **마루야마 공원**: 교토 최대의 하나미 장소로 야간 조명이 환상적입니다
- **철학자의 길**: 약 2km의 산책로를 따라 벚꽃이 늘어서 있습니다
- **기온 마츠리 준비**: 7월 축제를 위한 준비 과정을 볼 수 있습니다

## 🏔️ 여름 - 홋카이도 & 산간 지역

**홋카이도 여름 축제:**
- **삿포로 맥주 축제** (7-8월): 오도리 공원에서 열리는 대규모 맥주 축제
- **라벤더 축제** (7월): 후라노의 라벤더 밭에서 열리는 보라색 꽃 축제
- **요사코이 소란 축제** (6월): 삿포로의 대표적인 댄스 축제

## 🍁 가을 - 교토, 나라, 후지산 주변

**단풍 명소:**
- **교토 아라시야마**: 대나무 숲과 단풍의 조화가 아름다운 곳
- **나라 공원**: 사슴들과 함께 단풍을 즐길 수 있는 독특한 경험
- **가와구치 호수**: 후지산을 배경으로 한 단풍 리플렉션이 장관입니다

**가을 축제:**
- **지다이 마츠리** (10월 22일, 교토): 헤이안 시대부터 메이지 유신까지의 의상 행렬
- **단풍 라이트업**: 11월 중순~12월 초 교토 각 사찰에서 야간 조명

## ❄️ 겨울 - 하코네 & 홋카이도

**하코네 겨울 즐길거리:**
- **온천 료칸**: 눈 내리는 풍경을 보며 즐기는 노천온천
- **아시노코 호수**: 겨울 후지산의 설경이 호수에 반영되는 모습
- **하코네 신사**: 눈 덮인 도리이가 신비로운 분위기를 연출

**홋카이도 겨울

In [36]:
template_plan_final = """The following is a friendly conversation between a human and an AI. 
The AI is talkative and provides lots of specific details from its context. 

capitial :  {capital}
culture :   {culture}
climate :   {climate}

이 정보를 바탕으로 여행 계획을 세워주세요
"""
prompt_plan_final  = PromptTemplate(input_variables=["capital", "culture", "climate"],
                                 template=template_plan_final)

In [37]:
class TravelLast(BaseModel):
    capital: str = Field(description="국가의 수도에 대한 설명")
    culture: str = Field(description="국가의 문화에 대한 설명")
    climate : str = Field(description="국가 기후에 대한 설명")
    plan: str = Field(description="여행계획")

In [38]:
def transform_key_last(data):
    """Transform keys from history to chat_history"""
    return {
        "capital":  data.capital,
        "culture":  data.culture,  
        "climate":  data.climate, 
        "plan":     data.plan
    }

In [39]:
template_detail_final = """The following is a friendly conversation between a human and an AI. 
The AI is talkative and provides lots of specific details from its context. 

capitial :  {capital}
culture :   {culture}
climate :   {climate}
plan :      {plan}

plan에 담겨있는 관광지 혹은 축제에 대한 정보를 좀더 상세하게 알려두세요 
"""
prompt_detail_final = PromptTemplate(input_variables=["capital", "culture", "climate", "plan"],
                                 template=template_detail_final)

In [40]:
two_json_chain=({
    "capital": itemgetter("country") | capital_chain,
    "culture": itemgetter("country") | culture_chain,
    "climate": itemgetter("country") | climate_chain
    }
    | prompt_plan_final
    | llm.with_structured_output(TravelLast)
    | RunnableLambda(transform_key_last)
    | {
        "capital": itemgetter("capital"),   # capital("country") -> itemgetter("capital")
        "culture": itemgetter("culture"),   # 쉼표 추가
        "climate": itemgetter("climate"),   # 쉼표 추가
        "plan": itemgetter("plan")
    }
    | prompt_detail_final
    | llm
    | StrOutputParser()
    )


In [41]:
async for chunk in two_json_chain.astream({"country": "중국"}):
    print(chunk, end="")


네, plan에 언급된 관광지들에 대해 상세히 설명드리겠습니다!

## **베이징 관광지**

**자금성(紫禁城)**
- 명·청 황제들의 궁궐로 1420년 완공, 약 600년 역사
- 면적 72만㎡, 건물 980여 동, 방 9,999개의 거대한 규모
- 태화전, 중화전, 보화전 등 주요 전각들과 황제의 생활공간 관람 가능
- 입장료: 성인 60위안(비수기), 80위안(성수기)

**만리장성(八达岭 구간 추천)**
- 총 길이 2만여 km의 세계 최대 성벽, 베이징에서 차로 1시간 거리
- 팔달령 구간이 가장 잘 보존되어 관광객들이 많이 찾음
- 케이블카 이용 가능, 등반 시 2-3시간 소요
- 입장료: 40위안, 케이블카 왕복 140위안

**천단공원(天坛)**
- 명·청 황제들이 하늘에 제사를 지내던 제단
- 기년전(祈年殿)의 독특한 원형 건축이 유명
- 아침 일찍 가면 태극권하는 현지인들 구경 가능
- 입장료: 15위안, 통합권 35위안

## **상하이 관광지**

**와이탄(外滩)**
- 황푸강변 1.5km 구간의 서구식 건축물 거리
- 밤에는 푸둥 신구의 화려한 야경 감상 최적지
- 19-20세기 조계지 시절 은행, 호텔 건물들이 그대로 보존
- 무료 관람, 야경은 오후 7-9시가 가장 아름다움

**동방명주(东方明珠)**
- 높이 468m의 상하이 랜드마크 타워
- 3개의 구형 전망대에서 상하이 전경 조망
- 1층에는 상하이 역사박물관도 함께 운영
- 입장료: 전망대별로 120-220위안

**예원(豫园)**
- 명나라 시대(1559년) 조성된 전통 중국식 정원
- 연못, 정자, 기암괴석이 조화를 이룬 아름다운 경관
- 주변 예원상가에서 전통 간식과 기념품 쇼핑 가능
- 입장료: 40위안

## **주변 도시**

**소주(苏州) - "동양의 베니스"**
- 상하이에서 고속철로 30분 거리
- 졸정원, 유원, 망사원 등 9개 정원이 유네스코 세계문화유산
- 운하를 따라 발달한 수향 고진의 아름다운 풍경
- 정원 입장료: 각각 30-90위안

**항저